<div class='flex gap-1 items-center'>
    <img alt='self llama' src='../images/self-rag.jpeg' width='128' height='128' class='rounded'>
    <h1>Self Corrective RAG with LangGraph and Groq Llama 3</h1>
</div>

---


## Install dependencies

```bash
pip install langchain langchain-core langchain-community langchain-groq langgraph wikipedia
```


## Import libraries


In [34]:
from typing import Literal
from langchain_core.documents import Document
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_groq.chat_models import ChatGroq
from langchain_community.retrievers import WikipediaRetriever

## Basic RAG Components


### Language Model


In [3]:
llm = ChatGroq(
    model='llama3-groq-8b-8192-tool-use-preview'
)

res = llm.invoke('Hi! How are you today?')

res.content

"I'm doing well, thank you! How can I assist you today?"

### Documents Retriever


In [4]:
retriever = WikipediaRetriever(top_k_results=6)

docs = retriever.invoke("Meta AI")

docs[0].metadata

{'title': 'Meta AI',
 'summary': "Meta AI is an American company owned by Meta (formerly Facebook) that develops artificial intelligence and augmented and artificial reality technologies. Meta AI deems itself an academic research laboratory, focused on generating knowledge for the AI community, and should not be confused with Meta's Applied Machine Learning (AML) team, which focuses on the practical applications of its products.",
 'source': 'https://en.wikipedia.org/wiki/Meta_AI'}

In [5]:
print(docs[0].page_content)

Meta AI is an American company owned by Meta (formerly Facebook) that develops artificial intelligence and augmented and artificial reality technologies. Meta AI deems itself an academic research laboratory, focused on generating knowledge for the AI community, and should not be confused with Meta's Applied Machine Learning (AML) team, which focuses on the practical applications of its products. 

The laboratory was founded as Facebook Artificial Intelligence Research (FAIR) with locations at the headquarters in Menlo Park, California, London, United Kingdom, and a new laboratory in Manhattan. FAIR was officially announced in September 2013. FAIR was first directed by New York University's Yann LeCun, a deep learning professor and Turing Award winner. Working with NYU's Center for Data Science, FAIR's initial goal was to research data science, machine learning, and artificial intelligence and to "understand intelligence, to discover its fundamental principles, and to make machines sign

## Tools


### Format documents


In [6]:
def format_docs(docs: list[Document]) -> str:
    formatted = [
        (
            f"Source ID: {i+1}\n"
            f"Article Title: {doc.metadata['title']}\n"
            f"Article URL: {doc.metadata['source']}\n"
            f"Article Content: {doc.page_content}"
        )
        for i, doc in enumerate(docs)
    ]
    return "\n\n" + "\n\n".join(formatted)

In [7]:
print(format_docs(docs[:2]))



Source ID: 1
Article Title: Meta AI
Article URL: https://en.wikipedia.org/wiki/Meta_AI
Article Content: Meta AI is an American company owned by Meta (formerly Facebook) that develops artificial intelligence and augmented and artificial reality technologies. Meta AI deems itself an academic research laboratory, focused on generating knowledge for the AI community, and should not be confused with Meta's Applied Machine Learning (AML) team, which focuses on the practical applications of its products. 

The laboratory was founded as Facebook Artificial Intelligence Research (FAIR) with locations at the headquarters in Menlo Park, California, London, United Kingdom, and a new laboratory in Manhattan. FAIR was officially announced in September 2013. FAIR was first directed by New York University's Yann LeCun, a deep learning professor and Turing Award winner. Working with NYU's Center for Data Science, FAIR's initial goal was to research data science, machine learning, and artificial intel

## Chains


### Answer question with citations


#### CitedAnswer Output Model


In [8]:
class CitedAnswer(BaseModel):
    """Answer the user question based only on the given sources, and cite the sources used."""

    answer: str = Field(
        ...,
        description="The answer to the user question, which is based only on the given sources.",
    )
    citations: list[int] = Field(
        ...,
        description="The integer IDs of the SPECIFIC sources which justify the answer.",
    )

#### Prompt


In [9]:
RAG_SYSTEM_PROMPT = (
    "You are a helpful AI assistant. Your task is to answer questions based SOLELY on the information provided in the given articles.\n\n"
    "Main instructions:\n"
    "1. Answer the user's question using ONLY the information from the provided articles.\n"
    "2. Cite the source for EVERY statement you make, using the format [source_id] at the end of each sentence.\n"
    "3. If the information needed to answer the question is not in the provided articles, respond: \"I don't have enough information in the provided sources to answer this question.\"\n"
    "4. DO NOT use external knowledge or information not in the given articles, even if you believe it to be correct.\n"
    "5. If you can only partially answer the question, provide the information you have and then indicate that the rest of the question cannot be answered with the available information.\n"
    "6. Be concise and direct in your responses.\n\n"
    "Reference articles:\n"
    "{documents}\n\n"
)


RAG_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", RAG_SYSTEM_PROMPT),
        ("human", "{question}"),
    ]
)

RAG_PROMPT.pretty_print()

================================ System Message ================================

You are a helpful AI assistant. Your task is to answer questions based SOLELY on the information provided in the given articles.

Main instructions:
1. Answer the user's question using ONLY the information from the provided articles.
2. Cite the source for EVERY statement you make, using the format [source_id] at the end of each sentence.
3. If the information needed to answer the question is not in the provided articles, respond: "I don't have enough information in the provided sources to answer this question."
4. DO NOT use external knowledge or information not in the given articles, even if you believe it to be correct.
5. If you can only partially answer the question, provide the information you have and then indicate that the rest of the question cannot be answered with the available information.
6. Be concise and direct in your responses.

Reference articles:
{documents}



=========================

#### RAG Chain


In [10]:
rag_llm = llm.with_structured_output(CitedAnswer)

rag_response_chain = (
    RunnablePassthrough.assign(
        documents=(lambda x: format_docs(x["documents"]))
    )
    | RAG_PROMPT
    | rag_llm
)

retrieve_docs = (lambda x: x['question']) | retriever

rag_chain = RunnablePassthrough.assign(
    documents=retrieve_docs
).assign(response=rag_response_chain)

In [11]:
result = rag_chain.invoke({"question": "What is Meta AI?"})

result['response']

CitedAnswer(answer='Meta AI is an American company owned by Meta (formerly Facebook) that develops artificial intelligence and augmented and artificial reality technologies.', citations=[1])

In [12]:
result = rag_chain.invoke({"question": "Who is Jann LeCun?"})

result['response']

CitedAnswer(answer='Jann LeCun is a French computer scientist and director of AI Research at Facebook. He is known for his work on convolutional neural networks and is a co-recipient of the Turing Award.', citations=[1])

### Rewrite queries


#### Prompt


In [27]:
QUERY_REWRITER_SYSTEM = (
    "You are a query re-writer that converts an input query to an improved version optimized for Wikipedia search and retrieval. "
    "Your task is to generate ONE new query that is semantically related to the original and in the same domain. "
    "Do not provide explanations, ask questions, or add any text other than the new query. "
    "Analyze the input to understand its underlying intent, then output only the improved query."
)

QUERY_REWRITER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", QUERY_REWRITER_SYSTEM),
        (
            "human",
            "{query}",
        ),
    ]
)

QUERY_REWRITER_PROMPT.pretty_print()

================================ System Message ================================

You are a query re-writer that converts an input query to an improved version optimized for Wikipedia search and retrieval. Your task is to generate ONE new query that is semantically related to the original and in the same domain. Do not provide explanations, ask questions, or add any text other than the new query. Analyze the input to understand its underlying intent, then output only the improved query.

================================ Human Message =================================

{query}


#### Query Rewriter Chain


In [33]:
query_rewriter_chain = QUERY_REWRITER_PROMPT | llm | StrOutputParser()

result = query_rewriter_chain.invoke({"query": "What is Meta AI?"})

result

'Meta AI technologies'

### Check Context Relevance


#### CheckContextResponse Output Model


In [35]:
class CheckContextResponse(BaseModel):
    """Decide if the documents are relevant to the query."""

    binary_score: Literal["yes", "no"] = Field(
        description="Documents are relevants, 'yes' or 'no'"
    )

#### Prompt


In [36]:
CHECK_CONTEXT_SYSTEM = (
    "You are an evaluator for documents relevance. "
    "Given a set of documents and a query, you need to decide if the documents are relevant to the query. "
    "Your work is ask with `yes` if documents are relevant to the query or `no` if not. "
)

CHECK_CONTEXT_PROMPT = ChatPromptTemplate.from_messages(
    [
        ('system', CHECK_CONTEXT_SYSTEM),
        ('user', "DOCUMENTS:\n\n{documents}\n\nQUERY: {query}\n\n"),
    ]
)

CHECK_CONTEXT_PROMPT.pretty_print()

================================ System Message ================================

You are an evaluator for documents relevance. Given a set of documents and a query, you need to decide if the documents are relevant to the query. Your work is ask with `yes` if documents are relevant to the query or `no` if not. 

================================ Human Message =================================

DOCUMENTS:

{documents}

QUERY: {query}




#### Check Context Relevance Chain


In [37]:
check_context_llm = llm.with_structured_output(CheckContextResponse)
check_context_chain = CHECK_CONTEXT_PROMPT | check_context_llm

In [38]:
docs = retriever.invoke("Meta AI")

result = check_context_chain.invoke(
    {"documents": format_docs(docs), "query": "What is Meta AI?"}
)

result

CheckContextResponse(binary_score='yes')

In [43]:
result = check_context_chain.invoke(
    {"documents": format_docs(docs), "query": "Who is Miyamoto Musashi?"}
)

result

CheckContextResponse(binary_score='no')

### Check Hallucinations


#### CheckHallucinations Output Model


In [44]:
class CheckHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: Literal["yes", "no"] = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

#### Prompt


In [46]:
CHECK_HALLUCINATION_SYSTEM = (
    "You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. "
    "Give a binary score 'yes' or 'no', where 'yes' means that the answer is grounded in / supported by the set of facts. "
    "IF the generation includes code examples, make sure those examples are FULLY present in the set of facts, otherwise always return score 'no'. "
)

CHECK_HALLUCINATION_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", CHECK_HALLUCINATION_SYSTEM),
        ("human",
         "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

CHECK_HALLUCINATION_PROMPT.pretty_print()

================================ System Message ================================

You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. Give a binary score 'yes' or 'no', where 'yes' means that the answer is grounded in / supported by the set of facts. IF the generation includes code examples, make sure those examples are FULLY present in the set of facts, otherwise always return score 'no'. 

================================ Human Message =================================

Set of facts: 

 {documents} 

 LLM generation: {generation}


#### Check Hallucinations Chain


In [47]:
check_hallucination_llm = llm.with_structured_output(CheckHallucinations)

check_hallucination_chain = CHECK_HALLUCINATION_PROMPT | check_hallucination_llm

In [48]:
generation = 'Meta AI is an American company owned by Meta (formerly Facebook) that develops artificial intelligence and augmented and artificial reality technologies.'

result = check_hallucination_chain.invoke(
    {"documents": format_docs(docs), "generation": generation}
)

result

CheckHallucinations(binary_score='yes')

In [49]:
generation = 'Miyamoto Musashi was legendary Japanese hero, artist and swordsman.'

result = check_hallucination_chain.invoke(
    {"documents": format_docs(docs), "generation": generation}
)

result

CheckHallucinations(binary_score='no')

### Check Answer Quality

#### CheckAnswer Output Model

In [50]:
class CheckAnswer(BaseModel):
    """Binary score to assess answer addresses question."""
    binary_score: Literal["yes", "no"] = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

#### Prompt

In [53]:
CHECK_ANSWER_SYSTEM = (
    "You are a grader assessing whether an answer addresses / resolves a query. "
    "Give a binary score 'yes' or 'no', where 'yes' means that the answer resolves the query. "
    "Your main language is spanish"
)

CHECK_ANSWER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", CHECK_ANSWER_SYSTEM),
        ("human",
         "User query: {query}\n\nLLM generation: {generation}"),
    ]
)

CHECK_ANSWER_PROMPT.pretty_print()

================================ System Message ================================

You are a grader assessing whether an answer addresses / resolves a query. Give a binary score 'yes' or 'no', where 'yes' means that the answer resolves the query. Your main language is spanish

================================ Human Message =================================

User query: {query}

LLM generation: {generation}


#### Check Answer Chain

In [54]:
check_answer_llm = llm.with_structured_output(CheckAnswer)

check_answer_chain = CHECK_ANSWER_PROMPT | check_answer_llm

In [55]:
generation = 'Jann LeCun is a French computer scientist and director of AI Research at Facebook. He is known for his work on convolutional neural networks and is a co-recipient of the Turing Award.'

result = check_answer_chain.invoke(
    {"query": "What is Jann LeCun?", "generation": generation}
)

result

CheckAnswer(binary_score='yes')

In [57]:
result = check_answer_chain.invoke(
    {"query": "Who is Miyamoto Musashi?", "generation": generation}
)

result

CheckAnswer(binary_score='no')

## LangGraph Workflow

To be continued